In [11]:
import os
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
from tqdm import tqdm


ROOT_DIR = '../' if 'HMM' in os.getcwd() else os.getcwd() # setting the root dir
POS_DIR = os.path.join(ROOT_DIR, 'dataset') # setting the pos dir

pos_train = os.path.join(POS_DIR, "train.txt") 
pos_test = os.path.join(POS_DIR, "test.txt") 

In [14]:
def format_data(fname):
    sentences = [] # master list
    with open(fname) as f:
        content = f.readlines()
    
    sentence = [] # local list
    for line in content:
        if line !='\n':
            line = line.strip() # remove leading/trailing spaces
            word = line.split()[0].lower() # get the word
            pos = ""
            pos = line.split()[1] # get the pos tag
            sentence.append((word, pos)) # create a pair and save to local list
        else:
            sentences.append(sentence) # once a \n is detected, append the local sentence to master sentence
            sentence = []
    return sentences

train_set = format_data(pos_train)
test_set = format_data(pos_test)
print(len(train_set))
print(len(test_set))

8936
2012


In [13]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words), len(test_tagged_words))

211727 47377


In [15]:
tags = {tag for word,tag in train_tagged_words}
print(tags, len(tags))
vocab = {word for word,tag in train_tagged_words}

{'JJR', 'NNPS', 'NN', 'VB', 'CC', 'POS', '#', '$', 'VBD', 'EX', 'JJ', 'WDT', 'NNS', 'PRP$', 'PRP', 'SYM', 'FW', 'TO', '.', 'DT', 'RBS', ':', '(', 'CD', 'RP', 'WP$', 'PDT', 'RBR', '``', "''", 'VBG', 'VBP', 'NNP', 'UH', 'MD', ',', 'JJS', 'VBN', 'IN', ')', 'RB', 'WP', 'WRB', 'VBZ'} 44


In [16]:
def compute_emmision(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return count_w_given_tag /count_tag

transition_tags = [pair[1] for pair in train_tagged_words]


def compute_transition(t2, t1):
    count_t1 = len([t for t in transition_tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(transition_tags)-1):
        if transition_tags[index]==t1 and transition_tags[index+1] == t2:
            count_t2_t1 += 1
    return count_t2_t1/count_t1

In [17]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = compute_transition(t2, t1)
print(tags_matrix)

[[0.         0.00117233 0.27549824 ... 0.         0.00234467 0.00117233]
 [0.00238095 0.0047619  0.07619048 ... 0.01428571 0.         0.02619048]
 [0.00136    0.00029854 0.11772316 ... 0.00245464 0.00199025 0.03877666]
 ...
 [0.00189036 0.         0.02268431 ... 0.         0.         0.21550095]
 [0.00209205 0.0083682  0.07322176 ... 0.         0.         0.0041841 ]
 [0.00796041 0.         0.03958692 ... 0.00258176 0.00774527 0.00150602]]


In [18]:
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,JJR,NNPS,NN,VB,CC,POS,#,$,VBD,EX,...,MD,",",JJS,VBN,IN,),RB,WP,WRB,VBZ
JJR,0.000000,0.001172,0.275498,0.009379,0.022274,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.002345,0.036342,0.000000,0.002345,0.359906,0.001172,0.001172,0.000000,0.002345,0.001172
NNPS,0.002381,0.004762,0.076190,0.002381,0.078571,0.033333,0.000000,0.000000,0.121429,0.000000,...,0.035714,0.171429,0.000000,0.007143,0.104762,0.000000,0.004762,0.014286,0.000000,0.026190
NN,0.001360,0.000299,0.117723,0.001426,0.039705,0.022025,0.000033,0.000498,0.048628,0.000332,...,0.018178,0.112482,0.000000,0.012107,0.247753,0.002023,0.017050,0.002455,0.001990,0.038777
VB,0.013462,0.000831,0.065481,0.006814,0.011135,0.000166,0.000166,0.007977,0.001330,0.000831,...,0.000831,0.016952,0.000166,0.085092,0.139771,0.000000,0.045538,0.003158,0.005152,0.002493
CC,0.012472,0.002234,0.122487,0.033321,0.000000,0.000000,0.000000,0.025503,0.035741,0.004468,...,0.011355,0.006143,0.000186,0.021035,0.048771,0.000186,0.045421,0.002048,0.004095,0.021407
POS,0.001131,0.002261,0.419446,0.000000,0.004522,0.000000,0.001696,0.009610,0.005088,0.000000,...,0.000565,0.003957,0.022612,0.011871,0.002826,0.000565,0.007914,0.000000,0.000565,0.003957
#,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
$,0.000000,0.000000,0.009143,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000571,0.000000,0.000000,0.000000
VBD,0.009192,0.000000,0.043440,0.003410,0.003262,0.000000,0.000000,0.017643,0.001779,0.001483,...,0.000445,0.026093,0.000148,0.091179,0.136101,0.000148,0.077835,0.000593,0.001038,0.000890
EX,0.000000,0.000000,0.000000,0.000000,0.019417,0.000000,0.000000,0.000000,0.174757,0.000000,...,0.111650,0.033981,0.000000,0.004854,0.000000,0.000000,0.014563,0.000000,0.000000,0.436893


In [24]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in tqdm(enumerate(words),total=len(words)):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = compute_emmision(words[key], tag)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [21]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_tagged_words = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_run_untagged_words = [tup[0] for sent in test_run for tup in sent]


In [22]:
def test_accuracy(algorithm, tagged, untagged):
    start = time.time()
    tagged_seq = algorithm(untagged)
    end = time.time()
    difference = end-start
    
    print("Time taken in seconds: ", difference)
    
    # accuracy
    check = [i for i, j in zip(tagged_seq, tagged) if i == j] 
    
    accuracy = len(check)/len(tagged_seq)
    print('Viterbi Algorithm Accuracy: ',accuracy*100)
    return tagged_seq, check, accuracy

In [23]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
tagged_seq, check, accuracy = test_accuracy(Viterbi, test_run_tagged_words, test_run_untagged_words)


Time taken in seconds:  63.99494004249573
Viterbi Algorithm Accuracy:  90.9090909090909


In [25]:
#Code to test all the test sentences
#(takes alot of time to run s0 we wont run it here)
# tagging the test sentences()
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words

['rockwell',
 'international',
 'corp.',
 "'s",
 'tulsa',
 'unit',
 'said',
 'it',
 'signed',
 'a',
 'tentative',
 'agreement',
 'extending',
 'its',
 'contract',
 'with',
 'boeing',
 'co.',
 'to',
 'provide',
 'structural',
 'parts',
 'for',
 'boeing',
 "'s",
 '747',
 'jetliners',
 '.',
 'rockwell',
 'said',
 'the',
 'agreement',
 'calls',
 'for',
 'it',
 'to',
 'supply',
 '200',
 'additional',
 'so-called',
 'shipsets',
 'for',
 'the',
 'planes',
 '.',
 'these',
 'include',
 ',',
 'among',
 'other',
 'parts',
 ',',
 'each',
 'jetliner',
 "'s",
 'two',
 'major',
 'bulkheads',
 ',',
 'a',
 'pressure',
 'floor',
 ',',
 'torque',
 'box',
 ',',
 'fixed',
 'leading',
 'edges',
 'for',
 'the',
 'wings',
 'and',
 'an',
 'aft',
 'keel',
 'beam',
 '.',
 'under',
 'the',
 'existing',
 'contract',
 ',',
 'rockwell',
 'said',
 ',',
 'it',
 'has',
 'already',
 'delivered',
 '793',
 'of',
 'the',
 'shipsets',
 'to',
 'boeing',
 '.',
 'rockwell',
 ',',
 'based',
 'in',
 'el',
 'segundo',
 ',',
 'cal

In [26]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
    (r'(The|the|A|a|An|an)$', 'AT'),   # articles
    (r'.*able$', 'JJ'),                # adjectives
    (r'.*ness$', 'NN'),                # nouns formed from adjectives
    (r'.*ly$', 'RB'),                  # adverbs
    (r'.*s$', 'NNS'),                  # plural nouns
    (r'.*ing$', 'VBG'),                # gerunds
    (r'.*ed$', 'VBD'),                 # past tense verbs
    (r'.*', 'NN'),                     # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [27]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in tqdm(enumerate(words), total=len(words)):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = compute_emmision(words[key], tag)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
        state.append(state_max)
    return list(zip(words, state))

In [28]:
#test accuracy on subset of test data 
tagged_seq, check, accuracy = test_accuracy(Viterbi_rule_based, test_run_tagged_words, test_run_untagged_words)

100%|██████████| 231/231 [01:04<00:00,  3.58it/s]

Time taken in seconds:  64.50315952301025
Viterbi Algorithm Accuracy:  93.93939393939394


In [29]:
tagged_seq, check, accuracy = test_accuracy(Viterbi_rule_based, test_tagged_words, test_untagged_words)

  0%|          | 80/47377 [00:23<3:51:17,  3.41it/s]